In [1]:
import pandas as pd 
df=pd.read_csv(r"C:/Users/\user/Documents/fraud_transactions7.csv")
print(df['is_fraud'].value_counts())
df['is_fraud'].value_counts(normalize=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['hour'] = df['timestamp'].dt.hour
df['dayofweek'] = df['timestamp'].dt.dayofweek
df['month'] = df['timestamp'].dt.month
df.drop('timestamp', axis=1, inplace=True)
df.drop(['transaction_id', 'card_id','merchant'], axis=1, inplace=True)



is_fraud
0    32420
1     7580
Name: count, dtype: int64


In [39]:
df_encoded = pd.get_dummies(df, drop_first=True)


In [44]:
df_0 = df[df['is_fraud'] == 0]
df_1 = df[df['is_fraud'] == 1]

train_0 = df_0.sample(n=7000, random_state=42)
train_1 = df_1.sample(n=5200, random_state=24)

remaining_0 = df_0.drop(train_0.index)
remaining_1 = df_1.drop(train_1.index)

test_0 = remaining_0.sample(n=2000, random_state=89)
test_1 = remaining_1.sample(n=1827, random_state=19)

train_data = pd.concat([train_0, train_1]).sample(frac=1, random_state=7).reset_index(drop=True)
test_data = pd.concat([test_0, test_1]).sample(frac=1, random_state=8).reset_index(drop=True)

X_train = train_data.drop("is_fraud", axis=1)
y_train = train_data["is_fraud"]

X_test = test_data.drop("is_fraud", axis=1)
y_test = test_data["is_fraud"]


In [41]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import joblib
categorical_cols1 = ['transaction_type', 'transaction_category', 'location', 'gender']
numerical_cols1 = ['amount', 'hour', 'dayofweek']


In [28]:
preprocessor2 = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols1),
    ('num', StandardScaler(), numerical_cols1)
])
pipeline2 = Pipeline(steps=[
    ('preprocessing1', preprocessor2),
    ('classifier1', RandomForestClassifier(
        n_estimators=200,
        max_depth=10,
        class_weight='balanced',
        random_state=42
    ))
])


In [29]:
pipeline2.fit(X_train, y_train)


Pipeline(steps=[('preprocessing1',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['transaction_type',
                                                   'transaction_category',
                                                   'location', 'gender']),
                                                 ('num', StandardScaler(),
                                                  ['amount', 'hour',
                                                   'dayofweek'])])),
                ('classifier1',
                 RandomForestClassifier(class_weight='balanced', max_depth=10,
                                        n_estimators=200, random_state=42))])

In [30]:
y_pred = pipeline2.predict(X_test)

print("🔍 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))
print("📈 ROC AUC Score:", roc_auc_score(y_test, y_pred))


🔍 Confusion Matrix:
 [[1516  484]
 [ 209 1618]]

📊 Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.76      0.81      2000
           1       0.77      0.89      0.82      1827

    accuracy                           0.82      3827
   macro avg       0.82      0.82      0.82      3827
weighted avg       0.83      0.82      0.82      3827

📈 ROC AUC Score: 0.8218024083196497


In [45]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

# 1. Categorical sütunları tap və encoding et
categorical_cols = X_train.select_dtypes(include='object').columns

X_train_encoded = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
X_test_encoded  = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)

# 2. Test datasında sütun fərqləri ola bilər – uyğunlaşdırırıq
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

# 3. SMOTE ilə balanslaşdırma
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_encoded, y_train)

# 4. Modeli qur və öyrət
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_smote, y_train_smote)

# 5. Təxmin et və nəticələrə bax
y_pred = rf.predict(X_test_encoded)
y_proba = rf.predict_proba(X_test_encoded)[:, 1]

print("📊 Classification Report:")
print(classification_report(y_test, y_pred))

print("📈 ROC AUC Score:", roc_auc_score(y_test, y_proba))


c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


📊 Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.76      0.81      2000
           1       0.77      0.86      0.81      1827

    accuracy                           0.81      3827
   macro avg       0.81      0.81      0.81      3827
weighted avg       0.81      0.81      0.81      3827

📈 ROC AUC Score: 0.842409277504105


In [ ]:
import pandas as pd 
df=pd.read_csv(r"C:/Users/\user/Documents/fraud_transactions7.csv")
print(df['is_fraud'].value_counts())
df['is_fraud'].value_counts(normalize=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['hour'] = df['timestamp'].dt.hour
df['dayofweek'] = df['timestamp'].dt.dayofweek
df['month'] = df['timestamp'].dt.month
df.drop('timestamp', axis=1, inplace=True)
df.drop(['transaction_id', 'card_id','merchant'], axis=1, inplace=True)



In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

# 1. Categorical və numerical sütunları müəyyən et
categorical_cols = ['transaction_type', 'transaction_category', 'location', 'gender']
numerical_cols = ['amount', 'hour', 'dayofweek']

# 2. Column transformer – encoding + scaling
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ('num', StandardScaler(), numerical_cols)
])

# 3. SMOTE + Model daxil olmaqla pipeline
pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(
        n_estimators=200,
        max_depth=10,
        class_weight='balanced',
        random_state=42
    ))
])


In [4]:
df_0 = df[df['is_fraud'] == 0]
df_1 = df[df['is_fraud'] == 1]

train_0 = df_0.sample(n=7000, random_state=42)
train_1 = df_1.sample(n=5200, random_state=24)

remaining_0 = df_0.drop(train_0.index)
remaining_1 = df_1.drop(train_1.index)

test_0 = remaining_0.sample(n=2000, random_state=89)
test_1 = remaining_1.sample(n=1827, random_state=19)

train_data = pd.concat([train_0, train_1]).sample(frac=1, random_state=7).reset_index(drop=True)
test_data = pd.concat([test_0, test_1]).sample(frac=1, random_state=8).reset_index(drop=True)

X_train = train_data.drop("is_fraud", axis=1)
y_train = train_data["is_fraud"]

X_test = test_data.drop("is_fraud", axis=1)
y_test = test_data["is_fraud"]


In [5]:
# Modeli fit et
pipeline.fit(X_train, y_train)

# Proqnozlaşdırma
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Nəticələr
print("📊 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\n📋 Classification Report:\n", classification_report(y_test, y_pred))
print("📈 ROC AUC Score:", roc_auc_score(y_test, y_proba))


📊 Confusion Matrix:
 [[1521  479]
 [ 210 1617]]

📋 Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.76      0.82      2000
           1       0.77      0.89      0.82      1827

    accuracy                           0.82      3827
   macro avg       0.83      0.82      0.82      3827
weighted avg       0.83      0.82      0.82      3827

📈 ROC AUC Score: 0.8492443897099069


In [6]:
import joblib
joblib.dump(pipeline, "rf_model10_encoded.pkl")  

['rf_model10_encoded.pkl']